In [ ]:
# Import from local package
import sys
sys.path.append('../model')
import torch
import numpy as np

# Device configuration
device = torch.device("cpu")

In [ ]:
### import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter
mpl.style.use('classic')

In [ ]:
from models import SimpleCNN2Layer as Model
from collectdata import collect_data

In [ ]:
_, _, dataset_test = collect_data(
    "/data/schreihf/PvFinder/July_31_75000.npz",
    30000, 10000,
    device=device)

In [ ]:
name = 'July_31_75000_2layer_{}.pyt'
model = Model().to(device)
model.load_state_dict(torch.load(name.format(25)))
model.eval()

In [ ]:
def testNet1(model, batch_size):
    rms_list=[]
    diff2_list=[]
    rms_square=0
    F_Square_Diff=0
    Target_Bar_Position=0
    Predicted_Bar_Position=0
    Predicted_Probability_Frequency=0
    Target_Probability_Frequency=0
    succeed_count=0
    fail_count=0
    pic_fail=0
    False_Positive_count = 0
    total_probability_Output=0
    total_probability_Label =0
    test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size)
    N=0
    with torch.no_grad():
        for inputs, labels in test_loader:
            N=N+1
            print("N", N)
            
            #Forward pass
            outputs = model(inputs)
            print(outputs.size())
            print(outputs[0])
            for ii in range (4000):
                if labels[0][ii].item()>0:
                    print("labels", labels[0][ii].item())
                    print("outputs", outputs[0][ii].item())
                    
            inputss=torch.zeros([batch_size,4000])
            for i in range (batch_size):
                for j in range (4000):
                    inputss[i][j]=inputs[i][0][j]
           
            zvals = np.linspace(-100,300, 4000, endpoint=False) + 0.05

            for i in range (100,3900):
                if labels[0][i]> labels[0][i+1] and labels[0][i]> labels[0][i-1] and labels[0][i]>0.3:
                    print("i",i)
                    N_tar=0
                    position_predict_list=[]
                    frequency_predict_list=[]
                    for j in range(-5,5):
                        if outputs[0][i+j]> 0.01:
                            N_tar=N_tar+1
                    for j in range (-5,5):
                        if outputs[0][i+j]>0:
                            position_predict_list.append(zvals[i+j])
                            frequency_predict_list.append(outputs[0][i+j])
                            total_probability_Output+=outputs[0][i+j].item()
                            Predicted_Probability_Frequency+=outputs[0][i+j].item()*zvals[i+j]
                        total_probability_Label+=labels[0][i+j].item()
                        Target_Probability_Frequency+=labels[0][i+j].item()*zvals[i+j]
                        
                    Predicted_Bar_Position=Predicted_Probability_Frequency/total_probability_Output
                    Target_Bar_Position=Target_Probability_Frequency/total_probability_Label
        
                    if (total_probability_Output>0.2) and N_tar>=3:
                        succeed_count=succeed_count+1
                        diff2_list.append(Target_Bar_Position-Predicted_Bar_Position)
                        for jj in range (len(position_predict_list)):
                            rms_square+=frequency_predict_list[jj]*(position_predict_list[jj]-Predicted_Bar_Position)**2
                        rms=(rms_square/Target_Probability_Frequency)**(1/2)
                        rms_list.append(rms)
                        
                        print("Target centeriod:", Target_Bar_Position)
                        print("Predicted centeriod:", Predicted_Bar_Position)
                        print("Difference:",Target_Bar_Position-Predicted_Bar_Position)
                        diff2_list.append(Target_Bar_Position-Predicted_Bar_Position)
                        print("Succeed!")
                    
                        print ("i",i)
                    else:
                        fail_count=fail_count+1
                        print("Fail!")
                    print ("i",i)
                    if N % 1== 0:
                        Bin1=12
                        plt.rcParams["font.weight"] = "bold"
                        plt.rcParams["axes.labelweight"] = "bold"
                        fig,ax1 = plt.subplots()
                        ax1.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
                        fig.set_figheight(10)
                        fig.set_figwidth(15)
                        ax1.bar([round(zvals[k]-0.05,2) for k in range (i-Bin1, i+Bin1)] , [inputss[0][k]*2500 for k in range (i-Bin1, i+Bin1)],  width=0.1, alpha=0.3 ,color='g')
                        ax1.set_ylabel('Kernel Density', color= 'g',weight='bold', size= 18)
                        ax1.tick_params('y',colors='k',labelsize=18)
                        ax1.tick_params('x',colors='k',labelsize=18)
                        ax1.legend('Kernel Density' )
                        ax1.set_ylim(0,max([inputss[0][k]*2500 for k in range (i-Bin1, i+Bin1)])*1.2)
                        ax1.set_xlabel('z values [mm]', weight='bold', size= 18 )
                        ax2=ax1.twinx()                
                        p1=ax2.bar([round(zvals[k]-0.05,2) for k in range (i-Bin1, i+Bin1)] , [labels[0][k] for k in range (i-Bin1, i+Bin1)], width=0.1, alpha=0.6, color='b')
                        p2=ax2.bar([round(zvals[k]-0.05,2) for k in range (i-Bin1, i+Bin1)] , [outputs[0][k] for k in range (i-Bin1, i+Bin1)], width=0.1, alpha=0.6, color= 'r')
                        ax2.set_xlabel('z values [mm]',weight='bold', size= 18)
                        ax2.set_ylabel('Probability', color = 'r',weight='bold', size= 18)
                        ax2.set_ylim(0, max(0.8,1.2*max([outputs[0][k] for k in range (i-Bin1, i+Bin1)])))
                        ax2.legend((p1[0], p2[0]), ('Target', 'Predicted'),fontsize=18)
                        ax2.tick_params('y', colors = 'k',labelsize=18 )
                        ax2.tick_params('x', colors = 'k',labelsize=18 )
                        ax2.set_xlabel('z values [mm]')
                        plt.xlim(zvals[i-Bin1]-0.05,zvals[i+Bin1]-0.05)
                        fig.tight_layout()
                        plt.rcParams["font.weight"] = "bold"
                        plt.rcParams["axes.labelweight"] = "bold"
                        plt.show()
                        #fig.savefig('25000_'+str(Bin1) + '_bins_' + str(pic) + '.png')
                    
                        Bin2=25
                        fig2,ax3 = plt.subplots()
                        fig2.set_figheight(10)
                        fig2.set_figwidth(15)
                        ax3.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
                        ax3.bar([round(zvals[k]-0.05,2) for k in range (i-Bin2, i+Bin2)] , [inputss[0][k]*2500 for k in range (i-Bin2, i+Bin2)],  width=0.1, alpha=0.3 ,color='g')
                        ax3.set_ylabel('Kernel Density', color= 'g',weight='bold', size= 18)
                        ax3.tick_params('y', colors = 'k',labelsize=18)
                        ax3.tick_params('x',colors='k',labelsize=18)
                        ax3.legend('Kernel Density')
                        ax3.set_ylim(0,max([inputss[0][k]*2500 for k in range (i-Bin2, i+Bin2)])*1.2)
                        ax3.set_xlabel('z values [mm]',weight='bold', size= 18)
                        ax4=ax3.twinx()                
                        p1=ax4.bar([round(zvals[k]-0.05,2) for k in range (i-Bin2, i+Bin2)] , [labels[0][k] for k in range (i-Bin2, i+Bin2)], width=0.1, alpha=0.6, color='b')
                        p2=ax4.bar([round(zvals[k]-0.05,2) for k in range (i-Bin2, i+Bin2)] , [outputs[0][k] for k in range (i-Bin2, i+Bin2)], width=0.1, alpha=0.6, color= 'r')
                        ax4.set_xlabel('z values [mm]')
                        ax4.set_ylabel('Probability', color = 'r',weight='bold', size= 18)
                        ax4.set_ylim(0, max(0.8, 1.2*max([outputs[0][k] for k in range (i-Bin2, i+Bin2)])))
                        ax4.legend((p1[0], p2[0]), ('Target', 'Predicted'),fontsize=18)
                        ax4.tick_params('y', colors = 'k',labelsize=18 )
                        ax4.tick_params('x',colors='k',labelsize=18)
                        ax4.set_xlabel('z values [mm]',weight='bold', size= 18)
                        plt.xlim(zvals[i-Bin2]-0.05,zvals[i+Bin2]-0.05)
                        fig2.tight_layout()
                        plt.rcParams["font.weight"] = "bold"
                        plt.rcParams["axes.labelweight"] = "bold"
                        plt.show()
                    total_probability_Output= 0
                    Predicted_Probability_Frequency= 0
                    total_probability_Label= 0
                    Target_Probability_Frequency= 0
                    rms_square=0
                        
                        
                    
                
                                              
                
    np.save("Diffe_List_July_29_drop", diff2_list)
    np.save("RMS_Each_PV_List_July_29_drop", rms_list)
    print("succeed:", succeed_count)  
    print("Fail:", fail_count)
    

In [ ]:
testNet1(model, batch_size=1)